<a href="https://colab.research.google.com/github/21020718/KLTN_2025_TrangNTT/blob/main/%5BFinal%5DDataset2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Bộ dữ liệu thứ hai**

In [ ]:
%autosave 60
import warnings
warnings.filterwarnings('ignore')

Autosaving every 60 seconds


## Preprocessing

In [ ]:
# Set random seeds for reproducibility
import random
import numpy as np
import torch

# Define a fixed seed value
SEED = 42

# Set random seeds for all libraries
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)  # For GPU if available
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

import sklearn.preprocessing
import sklearn.preprocessing._data as data
import sys
sys.modules["sklearn.preprocessing.data"] = data

import pandas as pd
from numpy import concatenate
from math import sqrt

from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import normalized_mutual_info_score

def Scaler(data):

    """
        Scaler all feature to range(0,1)
        Arguments:
          data:  Pandas DataFrame of data
        Return:
          scaler: scaler
          scaledDf:Pandas DataFrame of scaled data
    """

    values = data.values
    values = values.astype('float32')
    scaler = MinMaxScaler(feature_range=(0,1))
    scaled = scaler.fit_transform(values)
    scaledDf = pd.DataFrame(scaled,columns=data.columns)
    return scaler,scaledDf


def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as a list or NumPy array.
        n_in: Number of lag observations as input (X).
        n_out: Number of observations as output (y).
        dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """

    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)

    agg.drop(agg.columns[-(df.shape[1]-1):],axis = 1,inplace=True)
    return agg



if __name__ == '__main__':

    # Load data with date column preserved
    # data = pd.read_csv("/content/drive/MyDrive/KLTN/data/VN2008-2020/VN2008-2020.csv", encoding='utf-8-sig')
    data = pd.read_csv("/content/drive/MyDrive/VN2008-2025.csv", encoding='utf-8-sig')

    # Store dates separately before dropping the column
    dates = data["date"].copy()
    data.drop(columns=["date"], inplace=True)
    data.fillna(0, inplace=True)

    scaler, scaledDf = Scaler(data)

    # Add dates back to scaled data for splitting
    scaledDf_with_dates = scaledDf.copy()
    scaledDf_with_dates['date'] = dates

    # Create time series features
    reframed = series_to_supervised(scaledDf, n_in=12)

    # Add dates back to reframed data (dates correspond to the target time)
    # Since we're using 12 months of history, the date for each row should be the last month in the sequence
    reframed_dates = dates.reset_index(drop=True)
    reframed_with_dates = reframed.copy()
    reframed_with_dates['date'] = reframed_dates

## Build model

In [ ]:
# # Giảm số lượng/GRU unit, dense unit, epochs và sử dụng batch size nhỏ hơn để huấn luyện nhanh hơn.
# model_types = ['multi-scale']
# lstm_unit = [128, 256, 512]
# gru_unit = [8, 16, 32]
# drop_rate = [0.1, 0.2]
# dense_unit = [16, 32, 64]
# batch_size_num = [2, 4]
# epochs = [100]

model_types = ['lstm','gru','hybrid']
lstm_unit = [128,256]
gru_unit = [8,16]
drop_rate = [0.1,0.2]
dense_unit = [32,64]
batch_size_num = [4]
epochs = [100]

# # Replace the current parameter definitions
# model_types = ['hybrid', 'sequential', 'stacked', 'bidirectional', 'cnn-rnn', 'multi-scale', 'transformer-rnn', 'ensemble', 'lstm', 'gru']
# lstm_unit = [128]
# gru_unit = [8]
# drop_rate = [0.1]
# dense_unit = [64]
# batch_size_num = [2]
# epochs = [100]

import pandas as pd
import numpy as np
from numpy import concatenate
import itertools
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

class AttentionGRU(nn.Module):
    def __init__(self, input_dim, time_steps, gru_units, dropout_rate, dense_units):
        super(AttentionGRU, self).__init__()
        self.input_dim = input_dim
        self.time_steps = time_steps

        # Attention layers
        self.attention1 = nn.Linear(time_steps, time_steps)
        self.attention2 = nn.Linear(time_steps, time_steps)
        self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax(dim=1)

        # GRU layer
        self.gru = nn.GRU(input_dim, gru_units, batch_first=True)

        # Fully connected layers
        self.flatten = nn.Flatten()
        self.dropout = nn.Dropout(dropout_rate)
        self.dense1 = nn.Linear(gru_units * time_steps, dense_units)
        self.tanh = nn.Tanh()
        self.dense2 = nn.Linear(dense_units, 1)
        self.final_activation = nn.Sigmoid()

    def forward(self, x):
        # x shape: (batch_size, time_steps, input_dim)
        batch_size = x.size(0)

        # Attention mechanism
        a = x.permute(0, 2, 1)  # (batch_size, input_dim, time_steps)
        a = self.attention1(a)  # (batch_size, input_dim, time_steps)
        a = self.sigmoid(a)
        a = self.attention2(a)  # (batch_size, input_dim, time_steps)
        a = self.softmax(a)
        a = a.permute(0, 2, 1)  # (batch_size, time_steps, input_dim)

        # Apply attention weights
        attention_mul = torch.mul(x, a)

        # GRU layer - output: (batch, seq, hidden_size)
        gru_out, _ = self.gru(attention_mul)

        # Flatten and Dense layers
        flattened = self.flatten(gru_out)
        dropout_out = self.dropout(flattened)
        dense1_out = self.tanh(self.dense1(dropout_out))
        output = self.final_activation(self.dense2(dense1_out))

        return output

class AttentionLSTM(nn.Module):
    def __init__(self, input_dim, time_steps, lstm_units, dropout_rate, dense_units):
        super(AttentionLSTM, self).__init__()
        self.input_dim = input_dim
        self.time_steps = time_steps

        # Attention layers
        self.attention1 = nn.Linear(time_steps, time_steps)
        self.attention2 = nn.Linear(time_steps, time_steps)
        self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax(dim=1)

        # LSTM layer
        self.lstm = nn.LSTM(input_dim, lstm_units, batch_first=True)

        # Fully connected layers
        self.flatten = nn.Flatten()
        self.dropout = nn.Dropout(dropout_rate)
        self.dense1 = nn.Linear(lstm_units * time_steps, dense_units)
        self.tanh = nn.Tanh()
        self.dense2 = nn.Linear(dense_units, 1)
        self.final_activation = nn.Sigmoid()

    def forward(self, x):
        # x shape: (batch_size, time_steps, input_dim)
        batch_size = x.size(0)

        # Attention mechanism
        a = x.permute(0, 2, 1)  # (batch_size, input_dim, time_steps)
        a = self.attention1(a)  # (batch_size, input_dim, time_steps)
        a = self.sigmoid(a)
        a = self.attention2(a)  # (batch_size, input_dim, time_steps)
        a = self.softmax(a)
        a = a.permute(0, 2, 1)  # (batch_size, time_steps, input_dim)

        # Apply attention weights
        attention_mul = torch.mul(x, a)

        # LSTM layer - output: (batch, seq, hidden_size)
        lstm_out, _ = self.lstm(attention_mul)

        # Flatten and Dense layers
        flattened = self.flatten(lstm_out)
        dropout_out = self.dropout(flattened)
        dense1_out = self.tanh(self.dense1(dropout_out))
        output = self.final_activation(self.dense2(dense1_out))

        return output

class HybridLSTM_GRU(nn.Module):
    def __init__(self, input_dim, time_steps, lstm_units, gru_units, dropout_rate, dense_units):
        super(HybridLSTM_GRU, self).__init__()
        self.input_dim = input_dim
        self.time_steps = time_steps

        # Attention layers
        self.attention1 = nn.Linear(time_steps, time_steps)
        self.attention2 = nn.Linear(time_steps, time_steps)
        self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax(dim=1)

        # LSTM and GRU layers
        self.lstm = nn.LSTM(input_dim, lstm_units, batch_first=True)
        self.gru = nn.GRU(input_dim, gru_units, batch_first=True)

        # Fully connected layers
        self.flatten = nn.Flatten()
        self.dropout = nn.Dropout(dropout_rate)
        # Combined size from both LSTM and GRU
        self.dense1 = nn.Linear((lstm_units + gru_units) * time_steps, dense_units)
        self.tanh = nn.Tanh()
        self.dense2 = nn.Linear(dense_units, 1)
        self.final_activation = nn.Sigmoid()

    def forward(self, x):
        # x shape: (batch_size, time_steps, input_dim)

        # Attention mechanism
        a = x.permute(0, 2, 1)  # (batch_size, input_dim, time_steps)
        a = self.attention1(a)  # (batch_size, input_dim, time_steps)
        a = self.sigmoid(a)
        a = self.attention2(a)  # (batch_size, input_dim, time_steps)
        a = self.softmax(a)
        a = a.permute(0, 2, 1)  # (batch_size, time_steps, input_dim)

        # Apply attention weights
        attention_mul = torch.mul(x, a)

        # LSTM and GRU layers
        lstm_out, _ = self.lstm(attention_mul)
        gru_out, _ = self.gru(attention_mul)

        # Concatenate LSTM and GRU outputs
        combined = torch.cat((lstm_out, gru_out), dim=2)

        # Flatten and Dense layers
        flattened = self.flatten(combined)
        dropout_out = self.dropout(flattened)
        dense1_out = self.tanh(self.dense1(dropout_out))
        output = self.final_activation(self.dense2(dense1_out))

        return output

def build_model(train_X, train_Y, val_X, val_Y, model_type='gru', lstm_units=128, gru_units=128, drop_rate=0.3, dense_unit=64, batch_size=32, epochs=100):
    # Print training parameters
    train_X_tensor = torch.FloatTensor(train_X)
    train_Y_tensor = torch.FloatTensor(train_Y.reshape(-1, 1))
    val_X_tensor = torch.FloatTensor(val_X)
    val_Y_tensor = torch.FloatTensor(val_Y.reshape(-1, 1))

    # Create datasets and dataloaders
    train_dataset = TensorDataset(train_X_tensor, train_Y_tensor)
    val_dataset = TensorDataset(val_X_tensor, val_Y_tensor)

    # Create reproducible DataLoaders with fixed seeds
    train_generator = torch.Generator()
    train_generator.manual_seed(SEED)
    val_generator = torch.Generator()
    val_generator.manual_seed(SEED)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, generator=train_generator)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, generator=val_generator)

    # Initialize model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    time_steps = train_X.shape[1]
    input_dim = train_X.shape[2]

    # Initialize model with fixed initial weights
    torch.manual_seed(SEED)

    if model_type == 'gru':
        model = AttentionGRU(input_dim, time_steps, gru_units, drop_rate, dense_unit).to(device)
    elif model_type == 'lstm':
        model = AttentionLSTM(input_dim, time_steps, lstm_units, drop_rate, dense_unit).to(device)
    elif model_type == 'hybrid':
        model = HybridLSTM_GRU(input_dim, time_steps, lstm_units, gru_units, drop_rate, dense_unit).to(device)
    else:
        raise ValueError(f"Unknown model type: {model_type}")

    # Initialize optimizer and loss function
    optimizer = optim.Adam(model.parameters())
    criterion = nn.L1Loss()  # MAE loss

    # Training loop
    best_val_loss = float('inf')
    patience_counter = 0
    patience = 5
    lr_factor = 0.01

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item()

        # Print progress
        print(f'Epoch {epoch+1}/{epochs}, Train Loss: {running_loss/len(train_loader):.4f}, Val Loss: {val_loss/len(val_loader):.4f}')

        # Learning rate schedule based on validation loss
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                for param_group in optimizer.param_groups:
                    param_group['lr'] *= lr_factor
                patience_counter = 0
                print(f'Reducing learning rate by factor of {lr_factor}')

    return model

def mean_absolute_percentage_error(y_true, y_pred):
    mask = y_true != 0
    mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100
    return mape

def walk_forward(train_X, train_Y, val_X, val_Y, test_X, test_Y, grid_search, scaler):
    r, f, c = test_X.shape
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    all_predictions = {}
    all_adjusted_predictions = {}
    all_ground_truths = {}

    # Create lists to store all evaluation results
    original_valuelists = []
    adjusted_valuelists = []

    for x in grid_search:
        history_x = np.array([x for x in train_X])
        history_y = np.array([y for y in train_Y])
        predictions = list()
        adjusted_predictions = list()
        groundtrue = list()

        # Extract model type first to determine how to unpack the rest
        model_type = x[0]

        # Create the appropriate config_key and extract parameters based on model type
        if model_type in ['hybrid']:
            # Hybrid model has 7 parameters
            model_type, lstm_unit_val, gru_unit_val, drop, dense, batch, epoch = x
            units = f"L{lstm_unit_val}_G{gru_unit_val}"  # For logging
            config_key = f"{model_type}_lstmUnit{lstm_unit_val}_gruUnit{gru_unit_val}_drop{drop}_dense{dense}_batch{batch}_epochs{epoch}"
        else:
            # LSTM and GRU models have 6 parameters
            model_type, units, drop, dense, batch, epoch = x
            config_key = f"{model_type}_unit{units}_drop{drop}_dense{dense}_batch{batch}_epochs{epoch}"

        print("\n" + "*"*50)
        print(f"Starting walk-forward validation with parameters:")
        print(f"Model Type: {model_type}, Units: {units}, Dropout: {drop}, Dense Units: {dense}")
        print(f"Batch Size: {batch}, Epochs: {epoch}")
        print(f"Device: {device}")
        print(f"Total test samples: {len(test_X)}")
        print("*"*50 + "\n")

        for i in range(len(test_X)):
            print(f"\nTest iteration {i+1}/{len(test_X)}")
            print(f"Current training set size: {history_x.shape[0]} samples")

            if model_type in ['hybrid']:
                model = build_model(history_x, history_y, val_X, val_Y, model_type=model_type,
                                lstm_units=lstm_unit_val, gru_units=gru_unit_val, drop_rate=drop,
                                dense_unit=dense, batch_size=batch, epochs=epoch)
            else:
                model = build_model(history_x, history_y, val_X, val_Y, model_type=model_type,
                                lstm_units=units if model_type == 'lstm' else 128,
                                gru_units=units if model_type == 'gru' else 128,
                                drop_rate=drop, dense_unit=dense, batch_size=batch, epochs=epoch)

            # Rest of the function remains the same
            model.eval()

            # Convert test data to tensor
            test_tensor = torch.FloatTensor(test_X[i].reshape(1, f, c)).to(device)

            # Predict
            with torch.no_grad():
                yhat = model(test_tensor).cpu().numpy()

            inv_yhat, inv_y = inverscale(yhat, test_X[i], test_Y[i], scaler)
            predictions.append(inv_yhat)
            groundtrue.append(inv_y)

            # Observation
            obs_x = test_X[i]
            obs_y = test_Y[i]

            history_x = np.append(history_x, [obs_x], axis=0)
            history_y = np.append(history_y, obs_y)

        # Store predictions and ground truth for this configuration
        all_predictions[config_key] = np.array(predictions).flatten()
        all_ground_truths[config_key] = np.array(groundtrue).flatten()

        original_valuelist = evalue(np.array(predictions).flatten(), np.array(groundtrue).flatten())
        original_valuelist['model_type'] = model_type
        original_valuelist['units'] = units
        original_valuelist['drop_rate'] = drop
        original_valuelist['dense_unit'] = dense
        original_valuelist['batch_size'] = batch
        original_valuelist['epochs'] = epoch

        # Append to the lists of results
        original_valuelists.append(original_valuelist)

    # Combine all results
    all_original_valuelist = pd.concat(original_valuelists, ignore_index=True)

    return all_original_valuelist, all_predictions, all_ground_truths

def evalue(yhat, inv_y):
    valuelist = {}
    DLM_rmse = sqrt(mean_squared_error(inv_y, yhat))
    valuelist.update({'RMSE': {'DLM': DLM_rmse}})
    DLM_mae = mean_absolute_error(inv_y, yhat)
    valuelist.update({'MAE': {'DLM': DLM_mae}})
    DLM_mape = mean_absolute_percentage_error(inv_y, yhat)
    valuelist.update({'MAPE': {'DLM': DLM_mape}})
    return pd.DataFrame(valuelist)

def inverscale(yhat, test_X, test_Y, scaler):
    feature = len(scaler.scale_)
    test_Y = np.array(test_Y)
    test_X = test_X[1, 0:feature]
    test_X = test_X.reshape(1, test_X.shape[0])

    if len(yhat.shape) == 1:
        yhat = yhat.reshape(len(yhat), 1)

    inv_yhat = concatenate((yhat, test_X[:, :-1]), axis=1)
    inv_yhat = scaler.inverse_transform(inv_yhat)
    inv_yhat = inv_yhat[:, 0]

    test_Y = test_Y.reshape(1, 1)
    inv_y = concatenate((test_Y, test_X[:, :-1]), axis=1)
    inv_y = scaler.inverse_transform(inv_y)
    inv_y = inv_y[:, 0]
    return inv_yhat, inv_y

if __name__ == '__main__':
    values = reframed.values
    reframed_with_dates_values = reframed_with_dates.values

    # Convert date strings to datetime objects
    dates = pd.to_datetime(reframed_with_dates['date'])

    # Create masks for each split according to the specified date ranges
    # train_mask = ((dates >= '2008-01-01') & (dates <= '2017-12-31')) | ((dates >= '2020-01-01') & (dates <= '2020-05-31'))
    # val_mask = (dates >= '2018-01-01') & (dates <= '2018-12-31')
    # test_mask = ((dates >= '2019-01-01') & (dates <= '2019-12-31')) | ((dates >= '2020-06-01') & (dates <= '2020-07-31'))
    train_mask = ((dates >= '2008-01-01') & (dates <= '2023-12-31'))
    val_mask = (dates >= '2024-01-01') & (dates <= '2024-12-31')
    test_mask = ((dates >= '2025-01-01') & (dates <= '2025-12-31'))

    # Extract values for train, validation, and test sets (excluding the date column)
    train_data = reframed.loc[train_mask].values
    val_data = reframed.loc[val_mask].values
    test_data = reframed.loc[test_mask].values

    # Split into X and Y
    train_X, train_Y = train_data[:, :-1], train_data[:, -1]
    val_X, val_Y = val_data[:, :-1], val_data[:, -1]
    test_X, test_Y = test_data[:, :-1], test_data[:, -1]

    # Reshape input to be 3D [samples, timesteps, features]
    train_X = train_X.reshape(train_X.shape[0], 12, int(train_X.shape[1]/12))
    val_X = val_X.reshape(val_X.shape[0], 12, int(val_X.shape[1]/12))
    test_X = test_X.reshape(test_X.shape[0], 12, int(test_X.shape[1]/12))

    # Modified grid search creation for all model types
    grid_search = []
    for model_type in model_types:
        if model_type == 'lstm':
            grid_search.extend(
                list(itertools.product([model_type], lstm_unit, drop_rate, dense_unit, batch_size_num, epochs))
            )
        elif model_type == 'gru':
            grid_search.extend(
                list(itertools.product([model_type], gru_unit, drop_rate, dense_unit, batch_size_num, epochs))
            )
        else:
            # All other models (hybrid, sequential, stacked, etc.) need both LSTM and GRU units
            grid_search.extend(
                list(itertools.product([model_type], lstm_unit, gru_unit, drop_rate, dense_unit, batch_size_num, epochs))
            )

    original_valuelist, all_predictions, all_ground_truths = walk_forward(train_X, train_Y, val_X, val_Y, test_X, test_Y, grid_search, scaler)

    # Group results by model type
    # gru_results = adjusted_valuelist[adjusted_valuelist['model_type'] == 'gru']
    # lstm_results = adjusted_valuelist[adjusted_valuelist['model_type'] == 'lstm']

    print("Results:")
    print(original_valuelist)

Streaming output truncated to the last 5000 lines.
Epoch 92/100, Train Loss: 0.0523, Val Loss: 0.0898
Epoch 93/100, Train Loss: 0.0534, Val Loss: 0.0898
Epoch 94/100, Train Loss: 0.0538, Val Loss: 0.0898
Reducing learning rate by factor of 0.01
Epoch 95/100, Train Loss: 0.0517, Val Loss: 0.0898
Epoch 96/100, Train Loss: 0.0519, Val Loss: 0.0898
Epoch 97/100, Train Loss: 0.0538, Val Loss: 0.0898
Epoch 98/100, Train Loss: 0.0537, Val Loss: 0.0898
Epoch 99/100, Train Loss: 0.0526, Val Loss: 0.0898
Reducing learning rate by factor of 0.01
Epoch 100/100, Train Loss: 0.0517, Val Loss: 0.0898

Test iteration 2/3
Current training set size: 181 samples
Epoch 1/100, Train Loss: 0.1775, Val Loss: 0.3198
Epoch 2/100, Train Loss: 0.1536, Val Loss: 0.3267
Epoch 3/100, Train Loss: 0.1221, Val Loss: 0.0891
Epoch 4/100, Train Loss: 0.0792, Val Loss: 0.0945
Epoch 5/100, Train Loss: 0.0696, Val Loss: 0.0914
Epoch 6/100, Train Loss: 0.0654, Val Loss: 0.0953
Epoch 7/100, Train Loss: 0.0703, Val Loss: 0.087

## Result

In [ ]:
original_valuelist

,RMSE,MAE,MAPE,model_type,units,drop_rate,dense_unit,batch_size,epochs
0,268846.258877,264327.500000,13.097408,lstm,128,0.1,32,4,100
1,208947.081990,182543.796875,8.915150,lstm,128,0.1,64,4,100
2,296711.634069,277411.781250,13.665507,lstm,128,0.2,32,4,100
3,287577.446459,274013.000000,13.514574,lstm,128,0.2,64,4,100
4,221825.744944,198950.875000,9.732743,lstm,256,0.1,32,4,100
5,202390.313760,174370.671875,8.497536,lstm,256,0.1,64,4,100
6,261022.530828,226467.296875,11.063681,lstm,256,0.2,32,4,100
7,222435.966013,200513.375000,9.818383,lstm,256,0.2,64,4,100
8,189052.706810,167609.671875,8.190430,gru,8,0.1,32,4,100
9,195718.393617,164602.171875,8.006814,gru,8,0.1,64,4,100


In [ ]:
all_predictions

{'lstm_unit128_drop0.1_dense32_batch4_epochs100': array([1760144.2, 1697628.4, 1767952.9], dtype=float32),
 'lstm_unit128_drop0.1_dense64_batch4_epochs100': array([1844453.9, 1851815.6, 1774807.1], dtype=float32),
 'lstm_unit128_drop0.2_dense32_batch4_epochs100': array([1658017.9, 1738303.9, 1790150.9], dtype=float32),
 'lstm_unit128_drop0.2_dense64_batch4_epochs100': array([1694429.1, 1731097. , 1771142.9], dtype=float32),
 'lstm_unit256_drop0.1_dense32_batch4_epochs100': array([1791929.9, 1833199.6, 1796725.9], dtype=float32),
 'lstm_unit256_drop0.1_dense64_batch4_epochs100': array([1800604.5, 1862156. , 1832835.5], dtype=float32),
 'lstm_unit256_drop0.2_dense32_batch4_epochs100': array([1809096.6, 1840975.6, 1689233.9], dtype=float32),
 'lstm_unit256_drop0.2_dense64_batch4_epochs100': array([1817046. , 1828541.6, 1771580.2], dtype=float32),
 'gru_unit8_drop0.1_dense32_batch4_epochs100': array([1824266.6, 1848320.2, 1843292.1], dtype=float32),
 'gru_unit8_drop0.1_dense64_batch4_epoch

In [ ]:
all_ground_truths

{'lstm_unit128_drop0.1_dense32_batch4_epochs100': array([2070466., 1893933., 2054309.], dtype=float32),
 'lstm_unit128_drop0.1_dense64_batch4_epochs100': array([2070466., 1893933., 2054309.], dtype=float32),
 'lstm_unit128_drop0.2_dense32_batch4_epochs100': array([2070466., 1893933., 2054309.], dtype=float32),
 'lstm_unit128_drop0.2_dense64_batch4_epochs100': array([2070466., 1893933., 2054309.], dtype=float32),
 'lstm_unit256_drop0.1_dense32_batch4_epochs100': array([2070466., 1893933., 2054309.], dtype=float32),
 'lstm_unit256_drop0.1_dense64_batch4_epochs100': array([2070466., 1893933., 2054309.], dtype=float32),
 'lstm_unit256_drop0.2_dense32_batch4_epochs100': array([2070466., 1893933., 2054309.], dtype=float32),
 'lstm_unit256_drop0.2_dense64_batch4_epochs100': array([2070466., 1893933., 2054309.], dtype=float32),
 'gru_unit8_drop0.1_dense32_batch4_epochs100': array([2070466., 1893933., 2054309.], dtype=float32),
 'gru_unit8_drop0.1_dense64_batch4_epochs100': array([2070466., 189